In [1]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=0654327c92133e4e995a6ff0decb2d74441f9d850e91eb2716da21f81ad2f1bc
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from os import listdir
from os.path import join
dirfiles = listdir("/content/drive/My Drive/MathIRData/ARQMathAgg/dataset_v2/")
print(dirfiles)

['queries_train.tsv', 'collection_train.tsv', 'triples_train.jsonl', 'qrel_train', 'queries_test.tsv', 'qrel_test', 'triples_test.jsonl', '.gitignore', 'collection_test.tsv', 'aggregates']


In [15]:
!pip install ir-measures

In [21]:
from pylatexenc.latex2text import LatexNodes2Text
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
import numpy as np
from os import listdir
from os.path import join
import re
import ir_measures
from ir_measures import nDCG, P, Judged, MRR, MAP, R
import itertools

import pandas as pd
import json

def main():
  parse_all_documents('/content/drive/My Drive/MathIRData/ARQMathAgg/dataset_v2/',
                      'collection_test.tsv', 'queries_test.tsv', 'aggregates/collection_agg_test.json')

def get_top_k_paragraph(tokenizer, model, paragraphs, query, k):
    if k > len(paragraphs):
      raise ValueError(f"k ({k}) cannot be greater than the number of paragraphs ({len(paragraphs)}).")
    # Turn latex into plaintext first
    query = LatexNodes2Text(math_mode='verbatim', strict_latex_spaces=True).latex_to_text(query)
    paragraphs = [(pid, LatexNodes2Text(math_mode='verbatim', strict_latex_spaces=True).latex_to_text(text)) for pid, text in paragraphs]

    results = []
    for id, (pid, paragraph) in enumerate(paragraphs):
      inputs = tokenizer.encode_plus(query, paragraph, return_tensors="pt", truncation=True)
      with torch.no_grad():
        outputs = model(**inputs)
        score = torch.nn.functional.softmax(outputs.logits, dim=-1)[0][1].item()
      results.append((id, pid, score))

    results.sort(key=lambda x: x[2], reverse=True)
    rel_scores = [score for _, _, score in results]
    top_k_results = results[:k]
    return top_k_results, rel_scores

def parse_all_documents(base_path, collection_name, queries_name, meta_name):
    dirfiles = listdir(base_path)
    print(dirfiles)

    collection_path = join(base_path, collection_name)
    queries_path = join(base_path, queries_name)
    meta_path = join(base_path, meta_name)
    qrelfile = '/content/drive/My Drive/MathIRData/ARQMathAgg/dataset_v2/qrel_test'
    eval_res_out_path = f'/content/drive/My Drive/MathIRData/ARQMathAgg/Evaluation/ALBERT/'

    k = 1

    try:
        with open(meta_path, 'r', encoding='utf-8') as file:
            meta = json.load(file)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    collection = {}
    with open(collection_path, 'r', encoding='utf-8') as file:
      for line in file:
        match = re.match(r"(\d+)\s+(.*)", line)
        if match:
          index = int(match.group(1))
          text = match.group(2).strip()
          collection[index] = text


    queries = {}
    with open(queries_path, 'r', encoding='utf-8') as file:
      for line in file:
        match = re.match(r"(\d+)\s+(.*)", line)
        if match:
          index = int(match.group(1))
          text = match.group(2).strip()
          queries[index] = text

    tokenizer = AutoTokenizer.from_pretrained("albert/albert-base-v2")
    model = AutoModelForSequenceClassification.from_pretrained("AnReu/albert-for-math-ar-base-ft")

    runfile = f'/content/drive/My Drive/MathIRData/ARQMathAgg/Evaluation/ALBERT/run'
    with open(runfile, 'w', encoding='utf-8') as f:
      for obj in meta:
        qid = obj['qid']
        pids = obj['pids']
        correct_idx = obj['corr_idx']
        paragraphs = [(pid, collection.get(pid)) for pid in pids]
        query = queries.get(qid)
        top_k, rel_scores = get_top_k_paragraph(tokenizer, model, paragraphs, query, k)
        for idx, (id, pid, score) in enumerate(top_k):
            f.write(f"{qid} Q0 {pid} {idx+1} {score} STANDARD\n")


    qrels = ir_measures.read_trec_qrels(qrelfile)
    runs = ir_measures.read_trec_run(runfile)
    cutoffs = [k]

    ndcg_measure = [nDCG(cutoff=cutoff) for cutoff in cutoffs]
    p_measure = [P(cutoff=cutoff) for cutoff in cutoffs]
    jugded_measure = [Judged(cutoff=cutoff) for cutoff in cutoffs]
    mrr_measure = [MRR(cutoff=cutoff) for cutoff in cutoffs]
    map_measure = [MAP(cutoff=cutoff) for cutoff in cutoffs]

    all_measures = list(itertools.chain(ndcg_measure, p_measure, jugded_measure, mrr_measure, map_measure))

    print("Evaluating on:")
    print(all_measures)

    eval_res = ir_measures.calc_aggregate(all_measures, qrels, runs)
    with open(f"{eval_res_out_path}/res.csv", 'w', encoding='utf-8') as f:
      f.write("Metric,Value\n")
      for measure in all_measures:
        f.write(f"{str(measure)},{eval_res[measure]}\n")


def parse_latex_into_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    latex_text = ''.join(lines)

    text = LatexNodes2Text(math_mode='verbatim', strict_latex_spaces=True).latex_to_text(latex_text)
    paragraphs = [para.strip() for para in text.split(".\n\n") if para.strip()]
    return paragraphs

if __name__=='__main__':
    main()



['queries_train.tsv', 'collection_train.tsv', 'triples_train.jsonl', 'qrel_train', 'queries_test.tsv', 'qrel_test', 'triples_test.jsonl', '.gitignore', 'collection_test.tsv', 'aggregates']
0/99
2461992 Q0 0 1 0.9997264742851257 STANDARD

1/99
2309227 Q0 2 1 0.9997496008872986 STANDARD

2/99
2017015 Q0 3 1 0.9995967745780945 STANDARD

3/99
955328 Q0 4 1 0.9997803568840027 STANDARD

4/99
1562252 Q0 7 1 0.9995365142822266 STANDARD

5/99
2416868 Q0 9 1 0.839702844619751 STANDARD

6/99
2103852 Q0 10 1 0.9997084736824036 STANDARD

7/99
2212375 Q0 12 1 0.9992941617965698 STANDARD

8/99
2599812 Q0 15 1 0.9997578263282776 STANDARD

9/99
384581 Q0 17 1 0.9997630715370178 STANDARD

10/99
164534 Q0 18 1 0.9983805418014526 STANDARD

11/99
3050853 Q0 20 1 0.9996416568756104 STANDARD

12/99
94359 Q0 22 1 0.9728226661682129 STANDARD

13/99
2322942 Q0 30 1 0.9997305274009705 STANDARD

14/99
1554993 Q0 31 1 0.9997138381004333 STANDARD

15/99
462006 Q0 33 1 0.9996705055236816 STANDARD

16/99
773547 Q0 34